In [74]:
'''
将原始小时数据重采样为每天数据或每月数据
'''
import pandas as pd
import numpy as np

In [2]:
DATA_DIR = '../../data/data.csv'

In [3]:
df  = pd.read_csv(DATA_DIR)

In [4]:
df.head()

,Source.Name,date,hour,type,北京,天津,石家庄,唐山,秦皇岛,邯郸,保定,张家口,承德,廊坊,沧州,衡水,邢台
0,china_cities_20150102.csv,20150102.0,23.0,CO_24h,1.38,1.82,2.98,2.14,1.16,3.37,4.66,1.37,1.21,1.02,2.10,1.83,3.13
1,china_cities_20150102.csv,20150102.0,23.0,CO,2.35,1.55,3.12,2.89,2.82,2.17,5.60,1.81,1.59,2.22,2.95,1.47,3.56
2,china_cities_20150102.csv,20150102.0,23.0,O3_8h_24h,38.00,39.00,38.00,46.00,48.00,25.00,52.00,55.00,59.00,37.00,41.00,50.00,30.00
3,china_cities_20150102.csv,20150102.0,23.0,O3_8h,15.00,12.00,18.00,16.00,25.00,13.00,39.00,23.00,19.00,14.00,16.00,33.00,14.00
4,china_cities_20150102.csv,20150102.0,23.0,O3_24h,50.00,52.00,57.00,61.00,60.00,45.00,67.00,64.00,76.00,44.00,60.00,66.00,46.00


In [25]:
df['tag']  = df.type.apply(lambda x: True if x in ['O3_24h','NO2_24h','SO2_24h','PM2.5_24h'] else False)

In [27]:
df = df[df.tag == True]

In [35]:
groups = df.groupby(['type','hour'])

In [36]:
dates = df.date.unique()
types = df.type.unique()
col_names = list(df.columns)

In [39]:
# 获取某一指标某天的数据
groups.get_group((types[0],23.0))

,Source.Name,date,hour,type,北京,天津,石家庄,唐山,秦皇岛,邯郸,保定,张家口,承德,廊坊,沧州,衡水,邢台,tag
4,china_cities_20150102.csv,20150102.0,23.0,O3_24h,50.0,52.0,57.0,61.0,60.0,45.0,67.0,64.0,76.0,44.0,60.0,66.0,46.0,True
349,china_cities_20150103.csv,20150103.0,23.0,O3_24h,23.0,22.0,13.0,17.0,16.0,31.0,53.0,45.0,37.0,11.0,39.0,43.0,17.0,True
709,china_cities_20150104.csv,20150104.0,23.0,O3_24h,30.0,13.0,7.0,33.0,39.0,22.0,35.0,47.0,29.0,18.0,26.0,54.0,12.0,True
889,china_cities_20150105.csv,20150105.0,23.0,O3_24h,71.0,49.0,36.0,63.0,35.0,15.0,51.0,68.0,86.0,43.0,39.0,59.0,23.0,True
1219,china_cities_20150106.csv,20150106.0,23.0,O3_24h,64.0,56.0,129.0,72.0,43.0,52.0,62.0,81.0,82.0,43.0,62.0,62.0,40.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924457,china_cities_20220329.csv,20220329.0,23.0,O3_24h,81.0,91.0,69.0,91.0,108.0,111.0,73.0,109.0,104.0,95.0,91.0,102.0,87.0,True
924817,china_cities_20220330.csv,20220330.0,23.0,O3_24h,61.0,95.0,61.0,94.0,105.0,74.0,61.0,90.0,84.0,90.0,106.0,102.0,52.0,True
925177,china_cities_20220331.csv,20220331.0,23.0,O3_24h,92.0,99.0,108.0,101.0,108.0,103.0,104.0,116.0,98.0,102.0,100.0,112.0,105.0,True
925537,china_cities_20220401.csv,20220401.0,23.0,O3_24h,92.0,123.0,114.0,113.0,112.0,130.0,121.0,104.0,106.0,133.0,117.0,120.0,123.0,True


In [ ]:
data = {type_:groups.get_group((type_,23.0)) for type_ in types}

In [ ]:
for type_,mdata in data.items():    
    mdata['date'] = pd.to_datetime(mdata['date'].apply(lambda x: str(int(x))))

In [ ]:
month_data = {type_:mdata.resample('M',on='date').mean() for type_,mdata in data.items()}
month_data['O3_24h']

In [75]:
year_data = {type_:mdata.resample('Y',on='date').mean() for type_,mdata in data.items()}

In [76]:
year_data['O3_24h']

,hour,北京,天津,石家庄,唐山,秦皇岛,邯郸,保定,张家口,承德,廊坊,沧州,衡水,邢台,tag
date,,,,,,,,,,,,,,,
2015-12-31,23.0,115.839650,91.541787,98.706052,113.876081,75.345821,95.717579,115.766571,109.766571,117.714697,104.306358,107.155620,120.230548,94.847262,1.0
2016-12-31,23.0,112.591160,100.340659,103.766484,113.607143,97.480769,108.785714,107.401099,115.620879,116.519231,110.774725,118.831956,125.741758,102.365385,1.0
2017-12-31,23.0,119.285714,124.505587,121.734637,127.432961,118.717877,123.905028,132.229050,123.019553,114.067039,128.122905,129.751397,121.846369,131.047486,1.0
2018-12-31,23.0,118.054131,126.508621,126.667622,123.468571,109.764368,131.730659,133.725714,121.846821,114.905714,117.977011,131.957020,127.997126,131.477143,1.0
2019-12-31,23.0,113.289017,124.209632,122.940510,117.356941,111.252125,128.575071,122.643059,113.257790,106.838068,119.971671,119.325779,121.317280,124.792614,1.0
2020-12-31,23.0,109.443836,117.772603,115.249315,116.682192,111.906849,120.663934,112.939891,109.608219,107.019126,117.082192,122.240437,123.172603,120.758904,1.0
2021-12-31,23.0,99.741047,107.988981,114.842975,107.812672,107.300275,117.848485,111.341598,107.247934,97.399449,109.330579,113.906336,116.655647,119.600551,1.0
2022-12-31,23.0,73.076087,80.989130,77.326087,81.836957,83.445652,81.282609,79.478261,88.728261,80.293478,79.086957,85.391304,86.347826,79.771739,1.0


In [78]:
# df.to_csv('../../data/data_clean.csv',index=False)